In [2]:
%%capture
!pip install -U langchain-ollama langchain-community chromadb

In [5]:
%%capture
# 폴더 내 파일 효율적으로 처리
!pip install unstructured

In [19]:
import os

def convert_and_save_new(src_dir, dest_dir):
    # 1. 대상 폴더가 없으면 생성
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
        print(f"새 폴더 생성됨: {dest_dir}")

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith(".txt"):
                src_path = os.path.join(root, file)
                
                # 원본의 하위 폴더 구조를 유지하고 싶다면 아래 주석 해제 (선택사항)
                # rel_path = os.path.relpath(root, src_dir)
                # target_subdir = os.path.join(dest_dir, rel_path)
                # if not os.path.exists(target_subdir): os.makedirs(target_subdir)
                # dest_path = os.path.join(target_subdir, file)
                
                dest_path = os.path.join(dest_dir, file)

                try:
                    # 2. 원본(CP949) 읽기
                    with open(src_path, 'r', encoding='cp949') as f:
                        content = f.read()
                    
                    # 3. 새 경로에 UTF-8로 저장
                    with open(dest_path, 'w', encoding='utf-8') as f:
                        f.write(content)
                    print(f"성공: {file} -> {dest_dir}")
                except Exception as e:
                    print(f"실패 ({file}): {e}")

# 실행
src = "./개역개정-text"    # 원본 폴더
dest = "./개역개정-utf8"  # 새로 만들 폴더
convert_and_save_new(src, dest)

새 폴더 생성됨: ./개역개정-utf8
성공: 2-18빌레몬서.txt -> ./개역개정-utf8
성공: 2-01마태복음.txt -> ./개역개정-utf8
성공: 1-22아가.txt -> ./개역개정-utf8
성공: 2-10에베소서.txt -> ./개역개정-utf8
성공: 1-15에스라.txt -> ./개역개정-utf8
성공: 1-19시편.txt -> ./개역개정-utf8
성공: 1-17에스더.txt -> ./개역개정-utf8
성공: 1-21전도서.txt -> ./개역개정-utf8
성공: 1-36스바냐.txt -> ./개역개정-utf8
성공: 1-03레위기.txt -> ./개역개정-utf8
성공: 2-12골로새서.txt -> ./개역개정-utf8
성공: 2-13데살로니가전서.txt -> ./개역개정-utf8
성공: 1-29요엘.txt -> ./개역개정-utf8
성공: 2-22베드로후서.txt -> ./개역개정-utf8
성공: 2-04요한복음.txt -> ./개역개정-utf8
성공: 2-16디모데후서.txt -> ./개역개정-utf8
성공: 1-39말라기.txt -> ./개역개정-utf8
성공: 1-27다니엘.txt -> ./개역개정-utf8
성공: 2-15디모데전서.txt -> ./개역개정-utf8
성공: 2-11빌립보서.txt -> ./개역개정-utf8
성공: 2-21베드로전서.txt -> ./개역개정-utf8
성공: 2-07고린도전서.txt -> ./개역개정-utf8
성공: 1-13역대상.txt -> ./개역개정-utf8
성공: 1-16느헤미야.txt -> ./개역개정-utf8
성공: 1-02출애굽기.txt -> ./개역개정-utf8
성공: 1-23이사야.txt -> ./개역개정-utf8
성공: 1-26에스겔.txt -> ./개역개정-utf8
성공: 2-19히브리서.txt -> ./개역개정-utf8
성공: 1-20잠언.txt -> ./개역개정-utf8
성공: 1-28호세아.txt -> ./개역개정-utf8
성공: 1-34나훔.txt -> ./개역개정-utf8

In [25]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# 1. 폴더 로드
path = "./개역개정-utf8" 
# 성경 파일들이 인코딩 문제(cp949 vs utf-8)를 일으킬 수 있으니 확인이 필요합니다.
loader = DirectoryLoader(
    path, 
    glob="**/*.txt", 
    loader_cls=TextLoader, 
    loader_kwargs={'encoding': 'utf-8'}
)
all_documents = loader.load()

# 2. 문서 분할 (성경은 한 절이 짧으므로 chunk_size를 400 정도로 추천)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, 
    chunk_overlap=40,
    separators=["\n\n", "\n", ".", " "] # 성경 구조에 맞게 줄바꿈 우선 분할
)
docs = text_splitter.split_documents(all_documents)
print(f"분할된 문서 조각 개수: {len(docs)}개")


분할된 문서 조각 개수: 5855개


In [26]:
# 3. Ollama EmbeddingGemma 모델
embeddings = OllamaEmbeddings(model="embeddinggemma:300m")

# 4. 벡터 스토어 생성 및 저장
# 데이터가 많으므로 처음 한 번만 실행하면 됩니다.
vectorstore = Chroma.from_documents(
    documents=docs, 
    embedding=embeddings,
    persist_directory="./bible_vector_db"
)
print("성경 전권 임베딩 및 DB 저장이 완료되었습니다.")

성경 전권 임베딩 및 DB 저장이 완료되었습니다.


In [ ]:
# 5. 검색 테스트
query = "태초에 하나님이 천지를 창조하시니라"
rel_docs = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(rel_docs):
    print(f"\n[결과 {i+1}] 출처: {doc.metadata['source']}")
    print(f"내용: {doc.page_content}...")